In [5]:
# stdlib
import sys
import os
import json
import signal
from filelock import FileLock
from itertools import islice
import logging

# relative import error with python modules and what not, the path needs to change according to cwd :)
project_root = os.path.dirname(os.path.dirname(os.path.abspath("./")))
if project_root not in sys.path:
    sys.path.append(project_root)

# ray
import ray

# autogen
from autogen import OpenAIWrapper

# a2a
from agents.agent import CodingAssistant
from agents.agent_conf import base_cfg
from agents.agent import marl
from lib.embeddings import get_db_connection

# import lib.eval.MLBench.MLBench_eval as mlb
# from agents.coordinator import Coordinator
# from agents.agent import marl

In [6]:
(id(id(id(id))))

140539747501456

In [8]:
# NOTE
tasks_path = "../../runs/a2a_gpt/indist_python_only.jsonl"

In [9]:
coding_agent = marl("MLBench_papers")[-1]

/home/turbouser44/miniconda3/envs/a3a/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:151: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details aboutthe new implementation.
  warn_deprecated(
/home/turbouser44/miniconda3/envs/a3a/lib/python3.11/site-packages/langchain_community/vectorstores/pgvector.py:328: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata.Please note that filtering operators have been changed when using JSOB metadata to be prefixed with a $ sign to avoid name collisions with columns. I

In [5]:
# Intellisense :D
coding_agent: CodingAssistant

In [10]:
def load_tasks(file_path: str):
    with open(file_path, "r") as file:
        for line in file:
            task = json.loads(line)
            task_description = task["task_description"]
            task_idx = task["task_idx"]

            yield {
                "task_description": task_description,
                "task_idx": task_idx,
            }

In [11]:
# avg words/prompt
# sum([len(t.split())for t in tasks])/len(tasks)
# below is more efficient than sum([i for i in j]) -> [] creates an unnecessary intermediate array
# whereas using purely the generator (which is valid because sum expects and iterable) yields the lens for sum to consume one at a time.
def avg_words_per_task_description(task_generator):
    total_words, task_count = map(sum, zip(*((len(task['task_description'].split()), 1) for task in task_generator)))
    return total_words / task_count if task_count > 0 else 0

In [12]:
len(list(load_tasks(tasks_path)))

315

In [15]:
# avg_words_per_task_description(load_tasks(tasks_path))

737.2095238095238

In [16]:
tasks = load_tasks(tasks_path)

In [17]:
class TimeoutException(Exception):
    pass


def handler(signum, frame):
    raise TimeoutException("Prompt skipped due to timeout")


signal.signal(signal.SIGALRM, handler)

<Handlers.SIG_DFL: 0>

In [18]:
def execute_with_gpt2(task_generator):
    client = OpenAIWrapper(config_list=base_cfg["config_list"])
    dbconn = get_db_connection("MLBench_papers")
    all_combined_stats = []
    with open("baseGPT4full.jsonl", "a") as f:

        # The previous attempt failed at task 93, so index 92
        # NOTE: ADDED ISLICE
        for task_desc in islice(task_generator, 92, None):
            vec_db_query = client.create(
                messages=[
                    {
                        "role": "user",
                        "content": f"Create a query for the following task:\n{task_desc['task_description']}\n"
                        "The query will be used to search a vector database with relevant research papers in it, keep the query short, useful, and relevant. "
                        "DO NOT RESPOND WITH ANYTHING BUT A QUERY",
                    }
                ],
                model="gpt-3.5-turbo-0125",
                cache_seed=33,
            )
            vec_db_query = vec_db_query.choices[0].message.content
            assert vec_db_query, "No query generated for vector database search."

            retrieved_excerpts = dbconn.similarity_search(vec_db_query)
            context = [d.page_content for d in retrieved_excerpts]
            formatted_context = "".join(
                [f"Context {i + 1}: {doc}" for i, doc in enumerate(context)]
            )

            exe_feedback = []
            attempts = 0

            while attempts < 5:
                messages = [
                    {
                        "role": "system",
                        "content": f"You are tasked with generating code based on research papers. "
                        f"Think through the process step by step and generate complete working code. "
                        f"Do not fill in functions with 'pass' or ellipses, and do not make assumptions without justification. "
                        f"Here is the code generated from your previous attempt:\n{exe_feedback[-1]['code'] if exe_feedback else ''}\n"
                        f"Here is the previous execution log:\n{exe_feedback[-1]['logs'] if exe_feedback else ''}",
                    },
                    {
                        "role": "user",
                        "content": task_desc["task_description"],
                    },
                    {
                        "role": "assistant",
                        "content": f"Here is retrieved information related to the task:\n{formatted_context}",
                    },
                ]

                try:
                    signal.alarm(120)
                    response = client.create(messages=messages, cache_seed=33)
                    generated_content = response.choices[0].message.content
                    assert generated_content
                    ef = coding_agent.detect_and_execute_code(generated_content)
                    exe_feedback.append(*ef)
                    attempts += 1
                    signal.alarm(0)
                except Exception as e:
                    exe_feedback.append(
                        {"code": generated_content, "exit_code": -1, "logs": str(e)}
                    )
                    attempts += 1

            exit_codes = [feedback["exit_code"] for feedback in exe_feedback]
            combined_stats = {
                "task_description": task_desc["task_description"],
                "exe_feedback": exe_feedback,
                "task_idx": task_desc["task_idx"],
                "exit_codes": exit_codes,
            }
            all_combined_stats.append(combined_stats)

            f.write(json.dumps(combined_stats) + "\n")
    return all_combined_stats

In [ ]:
# execute_with_gpt2(tasks)

In [20]:
FNAME = "baseGPT_indist_python_only.jsonl"
logger = logging.getLogger(__name__)

# Note: Do not set the signal handler at the global level in the worker processes
# We'll set it within the remote function if needed


@ray.remote
def process_task(task_desc):
    signal.signal(signal.SIGALRM, handler)
    # Recreate client, dbconn, and coding_agent within the function
    client = OpenAIWrapper(config_list=base_cfg["config_list"])
    dbconn = get_db_connection("MLBench_papers")

    coding_agent = marl("MLBench_papers")[-1]  # type: ignore
    coding_agent: CodingAssistant

    try:
        signal.alarm(300)

        vec_db_query_response = client.create(
            messages=[
                {
                    "role": "user",
                    "content": (
                        f"Create a query for the following task:\n{task_desc['task_description']}\n"
                        "The query will be used to search a vector database with relevant research papers in it, keep the query short, useful, and relevant. "
                        "DO NOT RESPOND WITH ANYTHING BUT A QUERY"
                    ),
                }
            ],
            model="gpt-3.5-turbo-0125",
            # cache_seed=33, #full
            cache_seed=89,  # ID Python ONLY
        )
        vec_db_query = vec_db_query_response.choices[0].message.content.strip()
        assert vec_db_query, "No query generated for vector database search."

        # Retrieve context from the vector database
        retrieved_excerpts = dbconn.similarity_search(vec_db_query)
        context = [d.page_content for d in retrieved_excerpts]
        formatted_context = "".join(
            [f"Context {i + 1}: {doc}\n" for i, doc in enumerate(context)]
        )

        exe_feedback = []
        attempts = 0
        generated_content = ""

        while attempts < 5:
            messages = [
                {
                    "role": "system",
                    "content": (
                        "You are tasked with generating code based on research papers. "
                        "Think through the process step by step and generate complete working code. "
                        "Do not fill in functions with 'pass' or ellipses, and do not make assumptions without justification. "
                        f"Here is the code generated from your previous attempt:\n{exe_feedback[-1]['code'] if exe_feedback else ''}\n"
                        f"Here is the previous execution log:\n{exe_feedback[-1]['logs'] if exe_feedback else ''}"
                    ),
                },
                {
                    "role": "user",
                    "content": task_desc["task_description"],
                },
                {
                    "role": "assistant",
                    "content": f"Here is retrieved information related to the task:\n{formatted_context}",
                },
            ]

            try:
                signal.alarm(120)  # Adjust per attempt timeout
                response = client.create(messages=messages, cache_seed=33)
                generated_content = response.choices[0].message.content.strip()
                assert generated_content
                # Execute the generated code
                ef = coding_agent.detect_and_execute_code(generated_content)
                exe_feedback.append(*ef)
                signal.alarm(0)
            except Exception as e:
                exe_feedback.append(
                    {"code": generated_content, "exit_code": -1, "logs": str(e)}
                )
            finally:
                attempts += 1
                signal.alarm(0)  # Reset the alarm

        exit_codes = [feedback["exit_code"] for feedback in exe_feedback]
        combined_stats = {
            "task_description": task_desc["task_description"],
            "exe_feedback": exe_feedback,
            "task_idx": task_desc["task_idx"],
            "exit_codes": exit_codes,
        }

        # Write the combined_stats to the file using file locking
        with FileLock(FNAME + ".lock"):
            with open(FNAME, "a") as f:
                f.write(json.dumps(combined_stats) + "\n")

        logger.info(f"Task {task_desc['task_idx']} processed successfully.")

    except TimeoutException:
        logger.warning(f"Task {task_desc['task_idx']} timed out.")
    except Exception as e:
        logger.error(f"Error processing task {task_desc['task_idx']}: {e}")
    finally:
        signal.alarm(0)  # Ensure the alarm is reset



In [24]:
ray.shutdown()
ray.init()
processed_task_ids = set()
try:
    with open(FNAME, "r") as f:
        for line in f:
            data = json.loads(line)
            processed_task_ids.add(data["task_idx"])
except FileNotFoundError:
    pass

tasks_to_process = [
    task for task in tasks if task["task_idx"] not in processed_task_ids
]

futures = [process_task.remote(task) for task in tasks_to_process]

ray.get(futures)

ray.shutdown()

2024-09-16 18:50:10,568	INFO worker.py:1774 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


In [1]:
import anthropic

client = anthropic.Anthropic()

In [4]:
s = client.messages.create(
            temperature=0,
            messages=[
                {
                    "role": "user",
                    "content": f"Create a query for the following task:"
                    "The query will be used to search a vector database with relevant research papers in it, keep the query short, useful, and relevant. "
                    "DO NOT RESPOND WITH ANYTHING BUT A QUERY".strip(),
                }
            ],
            model="claude-3-haiku-20240307",
            max_tokens=1000,
            # cache_seed=33, #full
            # cache_seed=89,  # ID Python ONLY
        )


In [18]:
s.content[0].text

'"machine learning techniques for natural language processing"'

In [41]:
system_message = (
    "You are tasked with generating code based on research papers. "
    "Think through the process step by step and generate complete working code. "
    "Do not fill in functions with 'pass' or ellipses, and do not make assumptions without justification. "
)
messages = [
    # {
    #     "role": "system",
    #     "content": "You are tasked with generating code based on research papers. "
    #     "Think through the process step by step and generate complete working code. "
    #     "Do not fill in functions with 'pass' or ellipses, and do not make assumptions without justification. "
    #     f"Here is the code generated from your previous attempt:\n{exe_feedback[-1]['code'] if exe_feedback else ''}\n"
    #     f"Here is the previous execution log:\n{exe_feedback[-1]['logs'] if exe_feedback else ''}",
    # },
    {
        "role": "user",
        "content": "oba saloba",
    },
    {
        "role": "assistant",
        "content": f"Here is retrieved information related to the task:\n{'print hello world '}".strip(),
    },
]

In [42]:
response = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    system=system_message,
    temperature=1,
    messages=messages,
    max_tokens=8192,
)

In [43]:
print(response.content[0].text)

 in python

Here is my step-by-step process and complete working code to accomplish this task:

1. We need to use Python's print function to output text to the console.
2. The text to print is "Hello, World!" (with capitalization and punctuation).
3. We'll use a simple print statement with the text as a string argument.

Here is the complete Python code:

```python
print("Hello, World!")
```

This code will print "Hello, World!" to the console when run. It uses no assumptions beyond standard Python syntax and doesn't require any additional functions or libraries.
